In [38]:
"""
Author: Ian Coleman
Purpose: Import and clean CSV data regarding broilers
"""

from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from datetime import datetime
from pyspark.sql.functions import col, udf
sqlContext = SQLContext(sc)
sc = SparkContext.getOrCreate()


conf = (SparkConf()
 .setMaster("local")
 .setAppName("Cleaner")
 .set("spark.executor.memory", "1g"))

In [3]:
#Info on setup

#Our Python version was 3.5
print('Your Python Version:', sc.pythonVer)
#Our SC version was 2.2
print('Your Spark Version:', sc.version)


Your Python Version: 3.5
Your Spark Version: 2.2.0


In [4]:
#Establish DB as data frame
df = sqlContext.read.load('FF_broilers_v2.csv', 
                      format='com.databricks.spark.csv', 
                      header='true', 
                      inferSchema='true')

In [9]:
#Non-essential stuff, mostly to try out dataFrame manipulation
# df.printSchema()
# df.show(2, truncate=True)
# df.columns
# df.describe('Customer Code').show()
# df2 = df.withColumnRenamed("Body Weight (g)", 'BodyWeightGrams')
# print(type(df))
# print(type(df2))
# df2.filter(df2.BodyWeightGrams == 44).count()
# df2.orderBy(df2.BodyWeightGrams.desc()).show(5)
# df2.describe('BodyWeightGrams').show()
# output_df.orderBy(output_df.BodyWeightGrams.desc())
# output_df = df2.withColumn("BodyWeightGrams",df2["BodyWeightGrams"].cast('float'))
# output_df.filter(output_df.BodyWeightGrams.isNotNull()).count()
# output_df.select('BodyWeightGrams').show(5)
df2 = df.withColumnRenamed("Body Weight (g)", 'BodyWeightGrams')
output_df = df2.withColumn("BodyWeightGrams",df2["BodyWeightGrams"].cast('float'))
df_cleaned = output_df.filter(output_df.BodyWeightGrams.isNotNull())

<class 'pyspark.sql.dataframe.DataFrame'>


In [10]:
#Create var of the db without rows lacking body weight or where body weight == 0
df_cleaned.count()

2032

In [17]:
csv1 = sqlContext.read.load('test_data.csv',
                           format='com.databricks.spark.csv',
                           header ='true',
                           inferSchema='true')

In [57]:
csv1.show()

+------+----------+------+
|Number|      Date|NatNum|
+------+----------+------+
|     1|   2017-01|   377|
|     2|      17-3|      |
|     3|17-03-2017|   822|
|     4|  Stringer| 1 073|
|     5|      null| 1 319|
|     6|03-07-2011|Strung|
|     7|  03/02/91|  null|
+------+----------+------+



In [36]:
csv1.describe('Date').show()

+-------+----------+
|summary|      Date|
+-------+----------+
|  count|         6|
|   mean|      null|
| stddev|      null|
|    min|03-07-2011|
|    max|  Stringer|
+-------+----------+



In [58]:
func =  lambda x: if x.length == 6: x
# csv1_floats = csv1.withColumn('Date', csv1['Date'].cast('date'))
csv1_floats = csv1.withColumn('test', func(col('Date')))

TypeError: 'tuple' object is not callable

In [56]:
print(type(csv1_floats))
csv1_floats.show()

<class 'pyspark.sql.dataframe.DataFrame'>
+------+----------+------+
|Number|      Date|NatNum|
+------+----------+------+
|     1|2017-01-01|   377|
|     2|      null|      |
|     3|      null|   822|
|     4|      null| 1 073|
|     5|      null| 1 319|
|     6|      null|Strung|
|     7|      null|  null|
+------+----------+------+

